In [1]:
! pip install mesa --quiet

In [2]:
# Librerías de simulación multiagentes
from mesa import Agent, Model  # Define los agentes y el modelo base de la simulación
from mesa.space import MultiGrid, SingleGrid  # Permite colocar los agentes en una cuadrícula (multi o individual)
from mesa.time import SimultaneousActivation  # Activa a todos los agentes simultáneamente en cada paso de la simulación
from mesa.datacollection import DataCollector  # Recolecta y organiza datos de la simulación para análisis

# Librerías matemáticas y generación de aleatoriedad
import itertools  # Proporciona herramientas para crear combinaciones y permutaciones
import random  # Permite generar números y secuencias aleatorias, útil para la variabilidad en simulaciones
import math  # Contiene funciones matemáticas básicas, como operaciones trigonométricas y logarítmicas

# Librerías de visualización y gráficos
import matplotlib.pyplot as plt  # Creación y personalización de gráficos
import matplotlib.patches as patches  # Añade formas como rectángulos o círculos a gráficos de matplotlib
from matplotlib.colors import ListedColormap  # Define mapas de color personalizados
import matplotlib.animation as animation  # Crea animaciones, útil para representar dinámicamente la simulación
plt.rcParams["animation.html"] = "jshtml"  # Configura la visualización de animaciones en formato HTML
import matplotlib  # Configuración adicional de matplotlib
matplotlib.rcParams["animation.embed_limit"] = 2**128  # Ajusta el límite de tamaño para incrustar animaciones

# Librerías de manipulación y análisis de datos
import numpy as np  # Proporciona funciones para manejo de matrices y álgebra lineal, común en análisis de datos
import pandas as pd  # Ofrece estructuras de datos como DataFrames, útiles para manipular grandes cantidades de datos
import time  # Proporciona funciones para trabajar con fechas y horas

# Librerías de comunicación en red y servidor HTTP
from http.server import BaseHTTPRequestHandler, HTTPServer  # Implementa un servidor HTTP básico para manejar solicitudes
import logging  # Registro de eventos y errores del servidor, útil para el diagnóstico
import json  # Proporciona funciones para manejar datos en formato JSON, útil para comunicación entre aplicaciones

import heapq

In [ ]:
class LuigiAgent(Agent):
    def __init__(self, unique_id, model):
        super().__init__(unique_id, model)
        self.action_points = 4
        self.saved_points = 0  # Puntos ahorrados de turnos anteriores
        self.carrying_portrait = None  # Para saber si está cargando un retrato verdadero

    def calculate_action_points(self):
        """Calcula los puntos de acción disponibles al inicio del turno."""
        self.action_points += self.saved_points
        if self.action_points > 8:  # No puede exceder 8 puntos
            self.action_points = 8
        self.saved_points = 0  # Los puntos ahorrados se usan al inicio del turno

    def move(self, direction):
        """Mueve al agente en una dirección, si tiene puntos suficientes."""
        if self.action_points >= 1:
            # Implementar lógica de movimiento aquí, según el modelo
            self.action_points -= 1
        else:
            print("No hay suficientes puntos de acción para moverse.")

    def open_close_door(self, door):
        """Abre o cierra una puerta."""
        if self.action_points >= 1:
            # Implementar lógica de abrir/cerrar puerta aquí
            self.action_points -= 1
        else:
            print("No hay suficientes puntos de acción para abrir/cerrar la puerta.")
    
    def break_wall(self, wall):
        """Rompe una pared."""
        if self.action_points >= 2:
            # Implementar lógica de romper pared aquí
            self.action_points -= 2
        else:
            print("No hay suficientes puntos de acción para romper la pared.")

    def knock_out_ghost(self, ghost):
        """Derriba un fantasma."""
        if self.action_points >= 1:
            # Implementar lógica para derribar al fantasma
            self.action_points -= 1
        else:
            print("No hay suficientes puntos de acción para noquear al fantasma.")

    def kill_ghost(self, ghost):
        """Mata un fantasma."""
        if self.action_points >= 2:
            # Implementar lógica para matar al fantasma
            self.action_points -= 2
        else:
            print("No hay suficientes puntos de acción para matar al fantasma.")

    def examine_portrait(self, portrait):
        """Examina un retrato."""
        if self.action_points >= 1:
            # Implementar lógica para examinar retrato
            self.action_points -= 1
        else:
            print("No hay suficientes puntos de acción para examinar el retrato.")

    def move_with_portrait(self, direction):
        """Se mueve cargando un retrato."""
        if self.carrying_portrait:
            if self.action_points >= 2:
                # Implementar lógica para moverse cargando el retrato
                self.action_points -= 2
            else:
                print("No hay suficientes puntos de acción para moverse cargando un retrato.")
        else:
            print("No está cargando ningún retrato.")

    def decide_to_save_points(self):
        """Decide si ahorrar puntos de acción basándose en la estrategia."""
        # Prioridad 1: Si está cargando un retrato, mover hacia la salida es crítico
        if self.carrying_portrait:
            return False  # No ahorra puntos, necesita moverse hacia la salida.

        # Prioridad 2: Hay fuego cerca de víctimas o áreas importantes
        fire_nearby = self.model.check_fire_nearby(self.pos)
        if fire_nearby:
            return False  # Actuar en fuego cercano antes de ahorrar puntos.

        # Prioridad 3: Si no hay acciones urgentes, ahorrar puntos
        if self.action_points <= 2:
            # Si hay pocos puntos restantes, es más estratégico ahorrar
            return True

        # Prioridad 4: Condiciones generales para ahorrar
        return True  # Por defecto, ahorrar puntos si no hay prioridades críticas.

    def end_turn(self):
        """Termina el turno, evaluando si debe ahorrar puntos."""
        if self.decide_to_save_points():
            self.saved_points = self.action_points
            if self.saved_points > 8:  # Limitar a un máximo de 8 puntos
                self.saved_points = 8

        else:
            self.saved_points = 0  # No ahorra puntos
        self.action_points = 0  # Reinicia los puntos para el siguiente turno

    def step(self):
        """Define el comportamiento del agente en cada paso."""
        self.calculate_action_points()

        # Priorizar acciones según el entorno
        while self.action_points > 0:
            # Aquí se puede incluir lógica de movimiento o interacción
            break
        # Finaliza el turno:
        self.end_turn()

In [ ]:
class Mansion(Model):
    def __init__(self, width, height, luigi, ghost, walls, doors, doors_connected, entrances, portrait):
        super().__init__()

In [ ]:
def get_grid(model):
    grid = []
    for y in range(model.height):
        row = []
        for x in range(model.width):
            pos = (x, y)
            if any(agent for agent in model.schedule.agents if agent.pos == pos):
                row.append(3)  # Representa la posición de un agente
            elif model.grid_values[pos] == 4:
                row.append(4)  # Representa una víctima
            elif model.grid_values[pos] == 3:
                row.append(3)  # Representa una falsa alarma
            elif model.grid_values[pos] == 2:
                row.append(2)  # Representa fuego
            elif model.grid_values[pos] == 1:
                row.append(1)  # Representa humo
            else:
                row.append(0)  # Celda vacía
        grid.append(row)
    return grid

In [ ]:
def get_walls(model):
    grid = []
    for y in range(model.grid.height):
        row = []
        for x in range(model.grid.width):
          row.append(model.grid_walls[(x, y)][0])
        grid.append(row)
    return grid

In [ ]:
class Mansion(Model):
    def __init__(self, walls, fake_alarms, victims, fires, doors, doors_connected, entrances):
        super().__init__()
        self.width = 8
        self.height = 6
        self.steps = 0
        self.schedule = SimultaneousActivation(self)
        self.walls = walls
        self.fake_alarms = fake_alarms
        self.victims = victims
        self.fires = [(r - 1, c - 1) for r, c in fires]
        self.doors = doors
        self.doors_connected = doors_connected
        self.entrances = [(r - 1, c - 1) for r, c in entrances]
        self.rescued = 0
        self.losses = 0
        self.damage = 0
        self.status = "In progress"
        self.grid_values = {(x, y): 0 for y in range(self.height) for x in range(self.width)}
        self.grid_walls = {(x, y): ["0000", "0000"] for y in range(self.height) for x in range(self.width)}
        self.initialize_walls()
        self.initialize_fires()
        self.initialize_POI()
        self.initialize_agents()
        self.datacollector = DataCollector(
            model_reporters={"Steps": "steps", "Status": "status", "Rescued": "rescued", "Losses": "losses"},
            agent_reporters={"Position": lambda a: a.pos if hasattr(a, "pos") else None}
        )
    def initialize_walls(self):
        for y, row in enumerate(self.walls):
            for x, cell in enumerate(row):
                if any(cell):  # Si hay una pared
                    self.grid_walls[(x, y)][0] = ''.join(map(str, cell))

    def initialize_fires(self):
        for pos in self.fires:
            self.grid_values[pos] = 2

    def initialize_POI(self):
        for pos in self.fake_alarms:
            self.grid_values[(pos[1] - 1, pos[0] - 1)] = 3  # Falsa alarma
        for pos in self.victims:
            self.grid_values[(pos[1] - 1, pos[0] - 1)] = 4  # Víctima

    def initialize_agents(self):
        for i, pos in enumerate(self.entrances):
            agent = LuigiAgent(f"Agent-{i}", self, "rescuer")
            self.schedule.add(agent)

    def more_poi(self):
        current_pois = len([v for v in self.grid_values.values() if v == 3 or v == 4])
        if current_pois < 3:
            while current_pois < 3:
                new_point = (random.randint(0, self.width - 1), random.randint(0, self.height - 1))
                if self.grid_values[new_point] == 0:
                    self.grid_values[new_point] = random.choice([3, 4])  # 3: Falsa alarma, 4: Víctima
                    current_pois += 1

    def fire(self):
        positions = [pos for pos, value in self.grid_values.items() if value in (0, 1, 2)]
        for pos in positions:
            if self.grid_values[pos] == 0:  # Aparece humo
                self.grid_values[pos] = 1
            elif self.grid_values[pos] == 1:  # Humo a fuego
                self.grid_values[pos] = 2
            elif self.grid_values[pos] == 2:  # Explosión
                neighbors = self.get_neighborhood(pos)
                for neighbor in neighbors:
                    if self.explosion(pos, neighbor):
                        self.damage(pos, neighbor)

    def get_neighborhood(self, pos):
        x, y = pos
        neighbors = []
        if x > 0: neighbors.append((x - 1, y))  # Izquierda
        if x < self.width - 1: neighbors.append((x + 1, y))  # Derecha
        if y > 0: neighbors.append((x, y - 1))  # Arriba
        if y < self.height - 1: neighbors.append((x, y + 1))  # Abajo
        return neighbors

    def explosion(self, origin, neighbor):
        direction = self.direction(origin, neighbor)
        return direction is not None and self.grid_walls[origin][0][direction] == "1"

    def direction(self, origin, neighbor):
        x_diff, y_diff = neighbor[0] - origin[0], neighbor[1] - origin[1]
        if x_diff == 0 and y_diff < 0: return 0  # Arriba
        if x_diff < 0 and y_diff == 0: return 1  # Izquierda
        if x_diff == 0 and y_diff > 0: return 2  # Abajo
        if x_diff > 0 and y_diff == 0: return 3  # Derecha

    def game_over(self):
        if self.losses >= 4 or self.damage >= 24:
            self.status = "Defeat"
            return True
        if self.rescued >= len(self.victims):
            self.status = "Victory"
            return True
        return False

    def step(self):
        if not self.game_over():
            self.steps += 1
            self.schedule.step()
            self.fire()
            self.more_poi()

In [2]:
def procesar_txt(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()

    # Procesar paredes
    WALLS = []
    index = 0
    for _ in range(6):  # 6 líneas de paredes
        row = []
        cells = lines[index].strip().split()
        for cell in cells:
            walls = [int(d) for d in cell]  # Cada celda se descompone en 4 dígitos
            row.append(walls)
        WALLS.append(row)
        index += 1

    # Procesar puntos de interés
    FAKE_ALARMS = []
    VICTIMS = []
    for _ in range(3):  # Máximo 3 líneas para puntos de interés
        r, c, marker_type = lines[index].strip().split()
        if marker_type == 'f':  # Falsa alarma
            FAKE_ALARMS.append((int(r), int(c)))
        elif marker_type == 'v':  # Víctima
            VICTIMS.append((int(r), int(c)))
        index += 1

    # Procesar marcadores de fuego
    FIRES = []
    for _ in range(10):  # 10 líneas de fuego
        r, c = map(int, lines[index].strip().split())
        FIRES.append((r, c))
        index += 1

    # Procesar marcadores de puertas
    DOORS = {}
    DOORS_CONNECTED = {}
    for _ in range(8):  # 8 líneas de puertas
        r1, c1, r2, c2 = map(int, lines[index].strip().split())
        r1, c1, r2, c2 = r1, c1, r2, c2
        DOORS[(r1, c1, r2, c2)] = (r1, c1, r2, c2)
        DOORS_CONNECTED[(r1, c1)] = (r2, c2)
        DOORS_CONNECTED[(r2, c2)] = (r1, c1)
        index += 1

    # Procesar puntos de entrada
    ENTRANCES = []
    for _ in range(4):  # 4 líneas de puntos de entrada
        r, c = map(int, lines[index].strip().split())
        ENTRANCES.append((r, c))
        index += 1

    return WALLS, FAKE_ALARMS, VICTIMS, FIRES, DOORS, DOORS_CONNECTED, ENTRANCES

# Ruta del archivo
file_path = './final.txt'

# Llamar a la función
WALLS, FAKE_ALARMS, VICTIMS, FIRES, DOORS, DOORS_CONNECTED, ENTRANCES = procesar_txt(file_path)

# Imprimir los resultados
print("Paredes:")
for row in WALLS:
    print(row)

print("\nFalsas alarmas:", FAKE_ALARMS)
print("\nVíctimas:", VICTIMS)
print("\nFuegos:", FIRES)
print("\nPuertas:", DOORS)
print("\nPuertas conectadas:", DOORS_CONNECTED)
print("\nEntradas:", ENTRANCES)

Paredes:
[[1, 1, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [1, 0, 0, 1], [1, 1, 1, 1], [1, 1, 0, 1], [1, 1, 0, 1]]
[[0, 1, 1, 0], [0, 0, 1, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 1], [1, 1, 1, 1], [0, 1, 0, 0], [0, 0, 0, 1]]
[[1, 1, 0, 0], [1, 0, 0, 0], [0, 0, 0, 1], [1, 1, 1, 0], [1, 0, 1, 1], [1, 1, 1, 0], [0, 0, 1, 0], [0, 0, 1, 1]]
[[0, 1, 0, 0], [0, 0, 0, 1], [0, 1, 0, 0], [1, 0, 1, 0], [1, 0, 1, 0], [1, 0, 1, 1], [1, 1, 1, 0], [1, 0, 1, 1]]
[[0, 1, 0, 0], [0, 0, 0, 0], [0, 0, 0, 1], [1, 1, 0, 0], [1, 0, 0, 1], [1, 1, 0, 0], [1, 0, 0, 0], [1, 0, 0, 1]]
[[0, 1, 1, 0], [0, 0, 1, 0], [0, 0, 1, 1], [0, 1, 1, 0], [0, 0, 1, 1], [0, 1, 1, 0], [0, 0, 1, 0], [0, 0, 1, 1]]

Falsas alarmas: [(2, 4)]

Víctimas: [(6, 7), (4, 8)]

Fuegos: [(1, 7), (4, 3), (5, 2), (1, 5), (2, 2), (2, 6), (6, 1), (1, 8), (2, 1), (6, 4)]

Puertas: {(1, 6, 1, 7): (1, 6, 1, 7), (2, 6, 2, 7): (2, 6, 2, 7), (3, 6, 4, 6): (3, 6, 4, 6), (4, 6, 4, 7): (4, 6, 4, 7), (4, 5, 5, 5): (4, 5, 5, 5), (4, 6, 5, 6): (4, 